In [1]:
# d6594
# The actual code to generate some Bach v2

# Method:
# - Only train on 1080_c01.mid
# - Experiment with different input resolutions (Speeding up the music)
# - Experiment with different model architectures
# - Overfit the model

beat_resolution = 12  # Standard of 24 for full 34 and 44 representation
bar_length = beat_resolution * 4
bars = 8
NOTES_PER_OCTAVE = 12
MIDI_INPUTS = 128  # Length the rolls pitches must be
MIDDLE_C = 64
num_notes_above_c = int(1.8 * NOTES_PER_OCTAVE)  # Centered around C, how many on either side
num_notes_below_c = int(2.5 * NOTES_PER_OCTAVE)
num_pitches = num_notes_above_c + num_notes_below_c
lookback = bar_length*4  # How far back can the model look? (Roughly four bars)
step = 1  # Skip through it a bit

%matplotlib inline
import os
from functions import Midi

# Load the functions dir
midi = Midi(middle_c=MIDDLE_C,
            midi_inputs=MIDI_INPUTS,
            num_notes_above_c=num_notes_above_c,
            num_notes_below_c=num_notes_below_c,
            beat_resolution=beat_resolution)


In [3]:
root = '/Users/James/PycharmProjects/deep-learning/music/lstm-generation-bach'
#root = '/home/ubuntu/deep-learning/music/lstm-generation-bach'
midi_dir = os.path.join(root, 'midi')
model_dir = os.path.join(root, 'models')
extended = midi.load_midi(midi_dir, over_ride_fnames=['goldberg.mid'], display=False)
x, y = midi.vectorise(extended, lookback=lookback, step=step)

Found 36 files: ['1080-c12.mid', 'jsbwv552.mid', 'bwv539_2.mid', 'bwv582.mid', 'bwv539_1.mid', 'bwv525-1.mid', 'jsbwv541.mid', '1080-c01.mid', 'bwv525-2.mid', 'jsbwv543.mid', 'bwv525-3.mid', '1080-c02.mid', 'bwv1028.mid', 'bwv1029.mid', '1080c02b.mid', 'bwv537.mid', 'jsbwv533.mid', 'jsbwv532.mid', 'bwv686.mid', 'bwv653.mid', 'bwv733.mid', 'bwv552p.mid', 'bwv552f.mid', 'bwv1027.mid', 'bwv539.mid', 'bwv530-2.mid', 'bwv529-3.mid', 'bwv529-2.mid', 'bwv588.mid', 'bwv530-3.mid', 'bwv530-1.mid', 'bwv529-1.mid', 'goldberg.mid', 'jsbwv549.mid', 'bwv0541p.mid', 'bwv0541f.mid']
Lower bound 34.
Upper bound 85.
Num pitches 51
Refined midi files ['goldberg.mid']
goldberg.mid input shape: (75335, 128)
...Refined down 77 dimensions with 289 note loss.
...Loss of 0.17 %
...Output shape: (75335, 51)
extended output shape (75335, 51)
75143 individual phrases.


In [3]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.LSTM(256, input_shape=(lookback, num_pitches), return_sequences=False, dropout=0.1, recurrent_dropout=0.3))
#model.add(layers.LSTM(64, dropout=0.1, recurrent_dropout=0.3, return_sequences=False))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_pitches, activation='sigmoid'))
model.summary()

model.compile(loss='categorical_crossentropy',  # categorical_crossentropy or mse
              optimizer=RMSprop(),
              metrics=['acc'])

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               315392    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 51)                3315      
Total params: 359,859
Trainable params: 359,859
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from time import time
first_start = time()
for i in range(5):
    print('>Epoch batch', i)
    start = time()
    history = model.fit(x, y,
                        epochs=20,
                        batch_size=128,
                        verbose=1)
    print('Took %s minutes.' % ((time()-first_start)/60).__round__(2))
    midi.save_model(model, model_dir, fname='lstm-v6.h5')
print('Full train took %s minutes.' % ((time()-first_start)/60).__round__(2))

>Epoch batch 0
Epoch 1/20
3552/3552 [==============================] - 63s 18ms/step - loss: 11.3009 - acc: 0.0470
Epoch 2/20
3552/3552 [==============================] - 57s 16ms/step - loss: 10.4588 - acc: 0.1391
Epoch 3/20
3552/3552 [==============================] - 54s 15ms/step - loss: 9.6087 - acc: 0.2123
Epoch 4/20
1280/3552 [=========>....................] - ETA: 34s - loss: 8.9792 - acc: 0.1742

In [4]:
from keras import models
model = midi.load_model(models, model_dir, fname='lstm-v6.h5')

Loaded model "lstm-v6.h5"


In [5]:
temps = [0.2, 0.4, 0.5, 0.6, 0.8]

outputs = midi.generate(model, extended, temperatures=temps, lookback=lookback, length=16)

midi.smart_save(outputs, display=False)

Generating roll with temp 0.2 and length 1536
Generating roll with temp 0.5 and length 1536
Generating roll with temp 0.5 and length 1536
Generating roll with temp 0.8 and length 1536
Generating roll with temp 0.8 and length 1536
Generating roll with temp 1.0 and length 1536
Generating roll with temp 1.2 and length 1536
Saving 5 tracks in one file "generated.mid".
Saved file "generated.mid".


/Users/James/PycharmProjects/deep-learning/music/lstm-generation-bach/functions.py:135: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature
